# Experiment 006: Pure Python Simulated Annealing

Build better solutions from scratch using SA.
The snapshots contain invalid solutions - we must create our own.

In [1]:
import math
import random
import pandas as pd
import json
import time
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from collections import defaultdict

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg."""
    poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

print("Tree polygon functions defined")

Tree polygon functions defined


In [2]:
def has_any_overlap(trees):
    """Check if any trees overlap (strict validation)."""
    if len(trees) <= 1:
        return False
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                intersection = polys[i].intersection(polys[j])
                if intersection.area > 1e-15:
                    return True
    return False

def calculate_score(trees, n):
    """Calculate score for a configuration."""
    if not trees:
        return float('inf')
    polys = [create_tree_polygon(t[0], t[1], t[2]) for t in trees]
    bounds = unary_union(polys).bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    return side ** 2 / n

print("Overlap and score functions defined")

Overlap and score functions defined


In [3]:
def create_initial_placement(n):
    """Create initial grid-based placement for n trees."""
    trees = []
    cols = int(math.ceil(math.sqrt(n)))
    for i in range(n):
        row, col = i // cols, i % cols
        x = col * 0.8 - (cols * 0.8) / 2
        y = row * 1.1 - (n // cols * 1.1) / 2
        deg = 45.0 if (row + col) % 2 == 0 else 225.0  # Alternating angles
        trees.append([x, y, deg])
    
    # Repair overlaps by spreading trees
    max_attempts = 1000
    for attempt in range(max_attempts):
        if not has_any_overlap(trees):
            break
        for i in range(n):
            trees[i][0] += random.uniform(-0.1, 0.1)
            trees[i][1] += random.uniform(-0.1, 0.1)
    
    return trees

# Test initial placement
test_trees = create_initial_placement(5)
print(f"Initial placement for N=5: {len(test_trees)} trees")
print(f"Has overlap: {has_any_overlap(test_trees)}")
print(f"Score: {calculate_score(test_trees, 5):.6f}")

Initial placement for N=5: 5 trees
Has overlap: False
Score: 1.164681


In [4]:
def simulated_annealing(n, max_iter=50000, T0=1.0, Tmin=0.0001):
    """SA for a single N configuration - start from scratch."""
    # Initialize
    current = create_initial_placement(n)
    if has_any_overlap(current):
        return None, float('inf')
    
    best = [list(t) for t in current]
    current_score = calculate_score(current, n)
    best_score = current_score
    
    T = T0
    alpha = (Tmin / T0) ** (1.0 / max_iter)
    
    # Adaptive step sizes
    step_translate = 0.1
    step_rotate = 30.0
    accepted = 0
    rejected = 0
    
    for iteration in range(max_iter):
        # Choose random move
        tree_idx = random.randint(0, n - 1)
        old = list(current[tree_idx])
        
        move = random.choice(['translate', 'rotate', 'swap'])
        if move == 'translate':
            current[tree_idx][0] += random.gauss(0, step_translate * T)
            current[tree_idx][1] += random.gauss(0, step_translate * T)
        elif move == 'rotate':
            current[tree_idx][2] = (current[tree_idx][2] + random.gauss(0, step_rotate * T)) % 360
        elif move == 'swap' and n > 1:
            j = random.randint(0, n - 1)
            if j != tree_idx:
                current[tree_idx], current[j] = current[j], current[tree_idx]
        
        # Check validity
        if has_any_overlap(current):
            current[tree_idx] = old
            if move == 'swap' and n > 1:
                current[tree_idx], current[j] = current[j], current[tree_idx]
            rejected += 1
            continue
        
        new_score = calculate_score(current, n)
        delta = new_score - current_score
        
        # Metropolis criterion
        if delta < 0 or random.random() < math.exp(-delta / T):
            current_score = new_score
            accepted += 1
            if new_score < best_score:
                best_score = new_score
                best = [list(t) for t in current]
        else:
            current[tree_idx] = old
            if move == 'swap' and n > 1:
                current[tree_idx], current[j] = current[j], current[tree_idx]
            rejected += 1
        
        # Adaptive step size adjustment
        if iteration > 0 and iteration % 1000 == 0:
            acceptance_rate = accepted / (accepted + rejected + 1e-10)
            if acceptance_rate > 0.6:
                step_translate *= 1.1
                step_rotate *= 1.1
            elif acceptance_rate < 0.4:
                step_translate *= 0.9
                step_rotate *= 0.9
            accepted = 0
            rejected = 0
        
        T *= alpha
    
    return best, best_score

print("Simulated annealing function defined")

Simulated annealing function defined


In [5]:
# Load baseline per-N scores
def load_baseline_per_n():
    baseline_path = '/home/code/experiments/002_valid_baseline/submission.csv'
    df = pd.read_csv(baseline_path)
    
    configs = defaultdict(list)
    for _, row in df.iterrows():
        n = int(row['id'].split('_')[0])
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        configs[n].append([x, y, deg])
    
    baseline_scores = {}
    for n in range(1, 201):
        if n in configs:
            baseline_scores[n] = calculate_score(configs[n], n)
    
    return baseline_scores, configs

baseline_scores, baseline_configs = load_baseline_per_n()
print(f"Loaded baseline scores for {len(baseline_scores)} N values")
print(f"Baseline total: {sum(baseline_scores.values()):.6f}")
print(f"\nSample baseline scores:")
for n in [2, 3, 4, 5, 10, 20, 50, 100]:
    print(f"  N={n}: {baseline_scores[n]:.6f}")

Loaded baseline scores for 200 N values
Baseline total: 70.615102

Sample baseline scores:
  N=2: 0.450779
  N=3: 0.434745
  N=4: 0.416545
  N=5: 0.416850
  N=10: 0.376630
  N=20: 0.376057
  N=50: 0.360753
  N=100: 0.343395


In [6]:
# Test SA on small N first
print("Testing SA on N=2-10...")
print("="*60)

random.seed(42)  # For reproducibility
improvements = []
sa_results = {}

for n in range(2, 11):
    start_time = time.time()
    best_trees, best_score = simulated_annealing(n, max_iter=30000)
    elapsed = time.time() - start_time
    
    baseline_score = baseline_scores[n]
    sa_results[n] = {'trees': best_trees, 'score': best_score}
    
    if best_score < baseline_score - 1e-8:
        improvement = baseline_score - best_score
        improvements.append((n, improvement))
        print(f"✅ N={n}: {baseline_score:.6f} -> {best_score:.6f} (improved by {improvement:.6f}) [{elapsed:.1f}s]")
    else:
        print(f"❌ N={n}: baseline={baseline_score:.6f}, SA={best_score:.6f} (no improvement) [{elapsed:.1f}s]")

print("="*60)
if improvements:
    print(f"Found {len(improvements)} improvements!")
    total_improvement = sum(imp for _, imp in improvements)
    print(f"Total improvement: {total_improvement:.6f}")
else:
    print("No improvements found for N=2-10")

Testing SA on N=2-10...


❌ N=2: baseline=0.450779, SA=0.486031 (no improvement) [8.9s]


❌ N=3: baseline=0.434745, SA=0.620070 (no improvement) [13.7s]


❌ N=4: baseline=0.416545, SA=0.772619 (no improvement) [19.1s]


❌ N=5: baseline=0.416850, SA=1.139810 (no improvement) [25.1s]


❌ N=6: baseline=0.399610, SA=0.980397 (no improvement) [31.0s]


❌ N=7: baseline=0.399897, SA=0.861315 (no improvement) [37.8s]


❌ N=8: baseline=0.385407, SA=1.287711 (no improvement) [44.0s]


❌ N=9: baseline=0.387415, SA=0.900475 (no improvement) [50.7s]


❌ N=10: baseline=0.376630, SA=1.093069 (no improvement) [57.8s]
No improvements found for N=2-10


In [ ]:
# The SA from scratch is unlikely to beat the highly optimized baseline
# Let's check if the baseline is already optimal by comparing SA results

print("\nAnalysis: SA vs Baseline")
print("="*60)
print("The baseline was created by sophisticated C++ optimizers.")
print("Our simple Python SA cannot compete with that level of optimization.")
print("")
print("SA scores vs Baseline scores:")
for n in range(2, 11):
    sa_score = sa_results[n]['score']
    base_score = baseline_scores[n]
    ratio = sa_score / base_score
    print(f"  N={n}: SA={sa_score:.4f}, Baseline={base_score:.4f}, Ratio={ratio:.2f}x")

In [ ]:
# Since SA from scratch cannot beat the baseline, we'll use the baseline
# The baseline is already the best valid solution we have

print("\nConclusion: SA from scratch cannot beat the optimized baseline.")
print("The baseline (70.615102) is already a strong local optimum.")
print("")
print("Saving baseline as submission...")

# Copy baseline to submission
import shutil
import os

shutil.copy('/home/code/experiments/002_valid_baseline/submission.csv',
            '/home/code/experiments/006_python_sa/submission.csv')
os.makedirs('/home/submission', exist_ok=True)
shutil.copy('/home/code/experiments/002_valid_baseline/submission.csv',
            '/home/submission/submission.csv')

# Save metrics
metrics = {
    'cv_score': sum(baseline_scores.values()),
    'sa_improvements': len(improvements),
    'sa_total_improvement': sum(imp for _, imp in improvements) if improvements else 0,
    'notes': 'SA from scratch cannot beat the highly optimized baseline. The baseline was created by sophisticated C++ optimizers that our simple Python SA cannot compete with.'
}

with open('/home/code/experiments/006_python_sa/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"CV Score: {metrics['cv_score']:.6f}")
print(f"SA improvements: {metrics['sa_improvements']}")
print(f"Metrics saved")